In [17]:
import os
import re
import math
import json
from collections import defaultdict
from tqdm import tqdm
import psutil

In [18]:
# Конфигурация
DATASET_DIR = 'C:/Users/szlat/Hates-speech-detection/dataset'
OUTPUT_FILE = 'embeddings.json'
MIN_WORD_COUNT = 15      # Минимальная частота слова
MAX_FEATURES = 2000      # Максимальный размер словаря
CHUNK_SIZE = 500         # Размер порции обработки
KEEP_TEXT = False        # Сохранять исходный текст

In [29]:
STOPWORDS = {'и', 'в', 'не', 'на', 'я', 'с'}

In [ ]:
def print_memory_usage(message):
    mem = psutil.virtual_memory()
    print(f"{message} - Используется памяти: {mem.used / (1024**3):.2f} GB")

In [30]:
def load_data(dataset_dir):
    """Загрузка данных с проверкой содержимого"""
    data = []
    labels_map = {'neg': 'negative', 'neu': 'neutral', 'pos': 'positive'}
    
    for label_dir in labels_map:
        dir_path = os.path.join(dataset_dir, label_dir)
        if not os.path.exists(dir_path):
            continue
            
        files = [f for f in os.listdir(dir_path) if f.endswith('.txt')]
        for filename in tqdm(files, desc=f'Loading {label_dir}'):
            file_path = os.path.join(dir_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                text = f.read().strip()
                if text:
                    data.append({
                        'text': text,
                        'label': labels_map[label_dir],
                        'filename': filename
                    })
    
    # Вывод примера для диагностики
    print("\nПример загруженного текста:")
    print(data[0]['text'][:100] + "...")
    return data

In [31]:
print("1. Загрузка данных...")
data = load_data(DATASET_DIR)
print(f"Загружено документов: {len(data)}")

1. Загрузка данных...


Loading pos: 100%|██████████| 87138/87138 [00:10<00:00, 8002.99it/s]


Пример загруженного текста:
В 2003-ем году под руководством малоизвестного режиссёра Кларка Джонсона студия 'Columbia Pictures' ...
Загружено документов: 131669


In [32]:
def preprocess_text(text):
    """Упрощенная предобработка с диагностикой"""
    words = re.findall(r'\b[а-яё]+\b', text.lower())  # Все слова без ограничения длины
    filtered = [w for w in words if w not in STOPWORDS]
    
    # Диагностика
    if len(filtered) == 0:
        print(f"\nПредупреждение: все слова отфильтрованы в тексте. Пример: {words[:5]}")
    
    return filtered

In [33]:
def build_vocabulary(data):
    """Построение словаря с подробной диагностикой"""
    word_counts = defaultdict(int)
    doc_samples = []
    
    for item in tqdm(data[:10000], desc="Анализ текстов"):  # Обрабатываем только часть для теста
        words = preprocess_text(item['text'])
        for word in words:
            word_counts[word] += 1
        if len(doc_samples) < 5 and words:
            doc_samples.append(words[:5])  # Сохраняем примеры слов
    
    print("\nПримеры найденных слов:")
    for i, sample in enumerate(doc_samples):
        print(f"Документ {i+1}: {sample}")
    
    # Анализ распределения частот
    freq_dist = sorted(word_counts.items(), key=lambda x: -x[1])
    print("\nТоп-10 самых частых слов:")
    for word, cnt in freq_dist[:10]:
        print(f"{word}: {cnt}")
    
    # Фильтрация
    filtered_words = {w for w, cnt in word_counts.items() if cnt >= MIN_WORD_COUNT}
    print(f"\nВсего слов до фильтрации: {len(word_counts)}")
    print(f"Слов после фильтрации (частотность >= {MIN_WORD_COUNT}): {len(filtered_words)}")
    
    if not filtered_words:
        print("\nКритическая ошибка: словарь пуст! Последние 10 слов:")
        print(freq_dist[-10:])
        return {}
    
    sorted_words = sorted(filtered_words, key=lambda x: -word_counts[x])[:MAX_FEATURES]
    return {word: idx for idx, word in enumerate(sorted_words)}

In [34]:
print("\n2. Построение словаря (тестовый режим)...")
vocabulary = build_vocabulary(data)
print(f"Размер словаря: {len(vocabulary)} слов")
    
if not vocabulary:
    print("\nОШИБКА: Не удалось построить словарь. Возможные причины:")
    print("- Все слова отфильтрованы стоп-словами")
    print("- Неправильная кодировка файлов (не UTF-8)")
    print("- Слишком высокий MIN_WORD_COUNT")
    print("- Ошибка в регулярном выражении для русских букв")


2. Построение словаря (тестовый режим)...


Анализ текстов: 100%|██████████| 10000/10000 [00:02<00:00, 4331.68it/s]


Примеры найденных слов:
Документ 1: ['ем', 'году', 'под', 'руководством', 'малоизвестного']
Документ 2: ['грустно', 'печально', 'грустно', 'от', 'того']
Документ 3: ['давным', 'давно', 'кира', 'найтли', 'ворвалась']
Документ 4: ['общем', 'ничего', 'против', 'уравновешенного', 'феминизма']
Документ 5: ['измена', 'один', 'из', 'сюжетов', 'который']

Топ-10 самых частых слов:
что: 52194
то: 31016
но: 30293
это: 28237
как: 27322
а: 26990
из: 23370
фильм: 21464
по: 16513
все: 16361

Всего слов до фильтрации: 185439
Слов после фильтрации (частотность >= 15): 18223
Размер словаря: 2000 слов


In [35]:
def process_chunk(chunk, vocabulary, idf):
    """Обработка порции данных"""
    embeddings = []
    for item in chunk:
        if item.get('text') is None:  # Пропускаем документы без текста
            embeddings.append([0.0]*len(vocabulary))
            continue
            
        words = preprocess_text(item['text'])
        total_words = len(words)
        
        if total_words == 0:
            embeddings.append([0.0]*len(vocabulary))
            continue
            
        word_counts = defaultdict(int)
        for word in words:
            if word in vocabulary:
                word_counts[word] += 1
        
        embedding = []
        for word in vocabulary:
            tf = word_counts.get(word, 0) / total_words
            embedding.append(tf * idf.get(word, 0))
        
        embeddings.append(embedding)
    return embeddings

In [36]:
print("\n3. Расчет IDF...")
    # Расчет IDF по всему датасету
doc_counts = defaultdict(int)
for item in tqdm(data, desc="Анализ документов"):
    if item.get('text') is not None:  # Только документы с текстом
        words = set(preprocess_text(item['text']))
        for word in words:
            if word in vocabulary:
                doc_counts[word] += 1
                    
idf = {word: math.log(len(data)/(cnt+1)) for word, cnt in doc_counts.items()}
    
print("\n4. Создание эмбеддингов...")
    # Обработка по частям
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    f.write('[')
    first_item = True
        
    for i in tqdm(range(0, len(data), CHUNK_SIZE), desc="Обработка"):
        chunk = data[i:i+CHUNK_SIZE]
        embeddings = process_chunk(chunk, vocabulary, idf)
            
        for item, emb in zip(chunk, embeddings):
            result = {
                'filename': item['filename'],
                'label': item['label'],
                'embedding': [float(f"{v:.4f}") for v in emb]
            }
            if KEEP_TEXT and item.get('text') is not None:
                result['text'] = item['text']
                
            if not first_item:
                f.write(',\n')
            json.dump(result, f, ensure_ascii=False)
            first_item = False
            
            # Очистка памяти
        del embeddings
        if i % (CHUNK_SIZE*2) == 0:
            print_memory_usage("Промежуточный отчет")
        
    f.write(']')
    
print_memory_usage("Завершение работы")
print(f"Результаты сохранены в {OUTPUT_FILE}")


3. Расчет IDF...


Анализ документов:  21%|██        | 27224/131669 [00:05<00:20, 5072.73it/s]


Предупреждение: все слова отфильтрованы в тексте. Пример: []

Предупреждение: все слова отфильтрованы в тексте. Пример: []


Анализ документов: 100%|██████████| 131669/131669 [00:28<00:00, 4673.27it/s]



4. Создание эмбеддингов...


...

Промежуточный отчет - Используется памяти: 13.13 GB


Обработка: 100%|█████████▉| 263/264 [07:10<00:01,  1.73s/it]

Промежуточный отчет - Используется памяти: 13.24 GB


Обработка: 100%|██████████| 264/264 [07:10<00:00,  1.63s/it]

Завершение работы - Используется памяти: 13.42 GB
Результаты сохранены в embeddings.json
